# AUTOBUS - Live demo

### Instructions
1. Check steer_model_file **(Rail Tracking Setup)** and oba_model **(Collision Avoidance Setup)** filenames are properly specified
2. Restart Kernel and Run all Cells
3. Go to **Display UI and Enable Controls** section at the bottom to operate AUTOBUS

In [1]:
steer_model_file = 'best_steering_model_xy--PoolTable400.pth'
oba_model_file = 'best_obstacle_model--NVIDIA.pth'

In [2]:
import time
print(time.ctime(),'go')

#####Make Robot#####
from jetbot import Camera, bgr8_to_jpeg
from jetbot import Robot
robot = Robot()

#####UI functions#####
import traitlets
import ipywidgets
import ipywidgets.widgets as widgets
from IPython.display import display

#####Monitor Network Connection#####
from jetbot import Heartbeat
def handle_heartbeat_status(change):
    if change['new'] == Heartbeat.Status.dead:
        print('check network connection')
heartbeat = Heartbeat(period=5)
heartbeat.observe(handle_heartbeat_status, names='status') # attach the callback function to heartbeat status

Mon Aug  3 12:56:46 2020 go


### Rail Tracking Setup

In [3]:
#####Rail Tracking Setup#####
import torchvision
import torch
#steer_model_file = 
steer_model = torchvision.models.resnet18(pretrained=False)
steer_model.fc = torch.nn.Linear(512, 2)
steer_model.load_state_dict(torch.load(steer_model_file))
steer_device = torch.device('cuda')
steer_model = steer_model.to(steer_device)
steer_model = steer_model.eval().half()
print(time.ctime(),steer_model_file,'ok')

import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np
steer_mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
steer_std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()
def steer_preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(steer_device).half()
    image.sub_(steer_mean[:, None, None]).div_(steer_std[:, None, None])
    return image[None, ...]

#add the steering xy prediction to the image
def display_xy(camera_image):
    image = np.copy(camera_image)
    x = x_slider.value
    y = y_slider.value
    x = int(x * 224 / 2 + 112)
    y = int(y * 224 / 2 + 112)
    image = cv2.circle(image, (x, y), 8, (0, 255, 0), 3)
    image = cv2.circle(image, (112, 224), 8, (0, 0,255), 3)
    image = cv2.line(image, (x,y), (112,224), (255,0,0), 3)
    jpeg_image = bgr8_to_jpeg(image)
    return jpeg_image

print(time.ctime(),'ok')

Mon Aug  3 12:57:04 2020 best_steering_model_xy--PoolTable400.pth ok
Mon Aug  3 12:57:04 2020 ok


### Collision Avoidance Setup

In [4]:
#####Collision Avoidance Setup#####
import torch
import torchvision

oba_model = torchvision.models.alexnet(pretrained=False)
oba_model.classifier[6] = torch.nn.Linear(oba_model.classifier[6].in_features, 2)
oba_model.load_state_dict(torch.load(oba_model_file))
oba_device = torch.device('cuda')
oba_model = oba_model.to(oba_device)
print(time.ctime(),oba_model_file,'ok')

import cv2
import numpy as np

oba_mean = 255.0 * np.array([0.485, 0.456, 0.406])
oba_stdev = 255.0 * np.array([0.229, 0.224, 0.225])
oba_normalize = torchvision.transforms.Normalize(oba_mean, oba_stdev)
def oba_preprocess(camera_value):
    global oba_device, oba_normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = oba_normalize(x)
    x = x.to(oba_device)
    x = x[None, ...]
    return x

import torch.nn.functional as F
import time

print(time.ctime(),'ok')

check network connection
Mon Aug  3 12:57:08 2020 best_obstacle_model--NVIDIA.pth ok
Mon Aug  3 12:57:08 2020 ok


### Create a function that will get called whenever the camera's value changes. 

In [5]:
##### Create a function that will get called whenever the camera's value changes. #####
#This function will do the following steps
#- Pre-process the camera image
#- Execute the neural network
#- Stop and prevent motion on obstacle detection
#- Stop when jetbot reaches a station, resumes after a set time
#- If OK to move, control steering, keep jetbot on the track

#####Global vars for execute#####
#all vars can be changed later, even while AUTOBUS is running
angle = 0.0
angle_last = 0.0
mt = 0
frame_cnt = 0
interval = 0.1
prob_block_stop = 0.8
tt = time.time()
obs_stop = False

#debug vars
steer_debug = False
fwd_high = 0.4
fwd_low = 0.3
turn_high = 0.4
turn_low = 0.3
stutter = 0
stutter_spd = 0.1

#station stopping vars
stopped_at_station = False #is it currently stopped at a station
time_left_station = 0 #the time (secs from epoch) it left last station
time_arrived_station = 0 #the time (secs from epoch) that it arrived at a station
station_stop_wait = 30 #stop at station for x secs
leave_station_span = 30 #the timespan (secs) it takes to leave a station (no red in sight)
f_red = 0.3
f_yellow = 0.1
f_green = 0.1

#Don't use time.sleep(), seems to break camera and other things
#!!!!! Jetbot doing something with new image !!!!!
def execute(change):
    global angle, angle_last, mt, frame_cnt, tt, steer_debug, interval, accel, stutter, robot
    global stutter_spd, prob_block_stop, min_inter, max_inter, fwd_high, fwd_low, turn_high, turn_low
    global stopped_at_station, time_left_station, time_arrived_station, station_stop_wait, leave_station_span, obs_stop
    global f_red, f_yellow, f_green
    
    #min time interval between each run
    if time.time() < tt + interval:
        return
    tt = time.time()
    
    #count images read
    frame_cnt += 1
    
    #hold at station
    if stopped_at_station:
        robot.left_motor.value = 0
        robot.right_motor.value = 0
        time_elapsed = time.time() - time_arrived_station
        status_a.value = str(frame_cnt) + " station stop " + str(int(time_elapsed)) + "/" + str(station_stop_wait) + " seconds"
        if time_elapsed > station_stop_wait: #leave station
            stopped_at_station = False
            time_left_station = time.time() #keep track of time since leaving station
        return
        
    #AAAAA jetbot reads image and makes a obstacle stop decision AAAAA
    x = change['new'] 
    x = oba_preprocess(x)
    y = oba_model(x)
    
    # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    y = F.softmax(y, dim=1)
    
    prob_blocked = float(y.flatten()[0])
    status_b.value = str(frame_cnt) + " obstacle prob: " + "{:.2f}".format(prob_blocked)
    if prob_blocked < 0.5:
        obs_stop = False
        
    elif prob_blocked > prob_block_stop:
        robot.left_motor.value = 0
        robot.right_motor.value = 0
        obs_stop = True
        status_a.value = str(frame_cnt) + " obstacle stop"
        
    #hold on obstacle
    if obs_stop:
        status_a.value = str(frame_cnt) + " obstacle stop"
        robot.left_motor.value = 0
        robot.right_motor.value = 0
        return
    #^^^^^ jetbot finishes obstacle stop decision ^^^^^

    #BBBBB jetbot reads image and makes a station stop decision BBBBB
    image = change['new']
    
    #red detection
    #cv2 hsv range for h is 0-180 instead of conventional 360, this caused issues and has been resolved
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV) 
    #lower_red1 = np.array([0,127,114]) 
    lower_red1 = np.array([0,127,114])
    #upper_red1 = np.array([15,204,166])
    upper_red1 = np.array([8,204,166])
    #lower_red2 = np.array([345,127,114])
    lower_red2 = np.array([173,127,114]) 
    #upper_red2 = np.array([360,204,166])
    upper_red2 = np.array([180,204,166])
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
    mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
    frac_red = sum(sum(mask1))/np.size(mask1)
    frac_red2 = sum(sum(mask2))/np.size(mask2)
    frac_red = frac_red+frac_red2
    
    #yellow detection
    #lower_yellow1 = np.array([45, 127, 102])
    lower_yellow1 = np.array([22, 127, 102])
    #upper_yellow1 = np.array([60, 255, 153])
    upper_yellow1 = np.array([30, 255, 153])
    mask3 = cv2.inRange(hsv, lower_yellow1, upper_yellow1)
    frac_yellow = sum(sum(mask3))/np.size(mask3)
    
    #green detection
    #lower_green1 = np.array([115, 102, 102])
    lower_green1 = np.array([57, 102, 102])
    #upper_green1 = np.array([150, 255, 153])
    upper_green1 = np.array([75, 255, 153])
    mask4 = cv2.inRange(hsv, lower_green1, upper_green1)
    frac_green = sum(sum(mask4))/np.size(mask4)
    
    #update sliders
    red_slider.value = frac_red
    yellow_slider.value = frac_yellow
    green_slider.value = frac_green
           
    #jetbot decides to make a station stop, or not 
    if frac_red > f_red:
        #dont't get stuck stopping in same station
        #if time since stopping at last station is more than time it takes to leave a station...
        if time.time() - time_left_station > leave_station_span: 
            robot.left_motor.value = 0
            robot.right_motor.value = 0
            stopped_at_station = True
            time_arrived_station = time.time()
            return
        
    elif frac_yellow > f_yellow:
        status_a.value = str(frame_cnt) + " near station slow"
        robot.left_motor.alpha = 0.5
        robot.right_motor.alpha = 0.5
        return
        
    elif frac_green > f_green:
        status_a.value = str(frame_cnt) + " resume full speed"
        robot.left_motor.alpha = 1
        robot.right_motor.alpha = 1
        
    #^^^^^ jetbot finishes station stop decision ^^^^^


    #CCCCC jetbot reads image and makes a steering decision CCCCC

    image = change['new']
    xy = steer_model(steer_preprocess(image)).detach().float().cpu().numpy().flatten()
    x = xy[0]
    y = (0.5 - xy[1]) / 2.0
    
    x_slider.value = x
    y_slider.value = y
    
    speed_slider.value = speed_gain_slider.value

    angle = np.arctan2(x, y)
    pid = angle * steering_gain_slider.value + (angle - angle_last) * steering_dgain_slider.value
    angle_last = angle

    steering_slider.value = pid + steering_bias_slider.value
   
    #nnnnn use default NVIDA algo nnnnn
    if not steer_debug:
        robot.left_motor.value = max(min(speed_slider.value + steering_slider.value, 1.0), 0.0) *mt
        robot.right_motor.value = max(min(speed_slider.value - steering_slider.value, 1.0), 0.0) *mt
        status_a.value = str(frame_cnt) + " move"
        return
        
    #xxxxx use AUTOBUS debugging algo xxxxx
    else:
        stutter = 1

        left_motor = max(min(speed_slider.value + steering_slider.value, 1.0), 0.0) 
        right_motor = max(min(speed_slider.value - steering_slider.value, 1.0), 0.0) 
        left_slider.value = left_motor
        right_slider.value = right_motor

        #run at preset speeds
        thres = turn_thres.value #steering value must exceed threshold to initate turning
        if steering_slider.value > thres:
            status_a.value = str(frame_cnt) + " turn right"
            robot.left_motor.value = min(robot.left_motor.value + 0.1, turn_high) *mt
            robot.right_motor.value = max(robot.right_motor.value - 0.1, turn_low) *mt
            stutter_spd = max(stutter_spd - 0.02, turn_low*0.9)
            
        elif steering_slider.value < -thres:
            status_a.value = str(frame_cnt) + " turn left"
            robot.left_motor.value = max(robot.left_motor.value - 0.1, turn_low) *mt
            robot.right_motor.value = min(robot.right_motor.value + 0.1, turn_high) *mt
            stutter_spd = max(stutter_spd - 0.02, turn_low*0.9)

        else: 
            status_a.value = str(frame_cnt) + " go straight"
            if robot.left_motor.value != robot.right_motor.value:
                robot.left_motor.value = robot.right_motor.value = min(robot.left_motor.value,robot.right_motor.value)
            #initial throttle then lower to coast speed    
            robot.left_motor.value = (robot.left_motor.value - 0.01) *mt
            robot.right_motor.value = (robot.right_motor.value - 0.01) *mt
            if min(robot.left_motor.value,robot.right_motor.value) < fwd_low:
                robot.left_motor.value = robot.right_motor.value = fwd_high *mt  
            stutter_spd = min(stutter_spd + 0.01, fwd_low*0.9)

        #set adjusted motor values shown in UI
        left_adjusted.value = robot.left_motor.value
        right_adjusted.value = robot.right_motor.value
        #^^^^^ jetbot finished steering decision ^^^^^
    
    return None
#^^^^^ jetbot image processing ^^^^^

print(time.ctime(),'ok')

Mon Aug  3 12:57:08 2020 ok


### Setup UI - camera view, buttons, etc.

In [42]:
#setup interface - camera view, buttons, etc
#
#user controlled sliders
#1. Speed Control (speed_gain_slider): To start your JetBot increase ``speed_gain_slider`` 
#2. Steering Gain Control (steering_gain_sloder): If you see JetBot is woblling, you need to reduce ``steering_gain_slider`` till it is smooth
#3. Steering Bias control (steering_bias_slider): If you see JetBot is biased towards extreme right or extreme left side of the track, you should control this slider till JetBot start following line or track in the center.  This accounts for motor biases as well as camera offsets
#jetbot turns when steering value exceeds threshold (for AUTOBUS steering algo only, not in NVIDIA steering algo)
speed_gain_slider = ipywidgets.FloatSlider(min=0, max=1, step=0.01, value=0.3, description='speed gain')
steering_gain_slider = ipywidgets.FloatSlider(min=0, max=1, step=0.01, value=0.1, description='steering gain')
steering_dgain_slider = ipywidgets.FloatSlider(min=0, max=1, step=0.01, value=0.1, description='steering kd')
steering_bias_slider = ipywidgets.FloatSlider(min=-1, max=1, step=0.01, value=0, description='steering bias')
turn_thres = ipywidgets.FloatSlider(min=0, max=1, step=0.01, value=0.25, description='steer thres') #turn threshold
#
#buttons
button_layout = ipywidgets.Layout(width='160px', height='35px')
start_button = ipywidgets.Button(description='START', button_style='success', layout=button_layout)
start_button.on_click(lambda x: start())
#
stop_button = ipywidgets.Button(description='STOP', button_style='danger', layout=button_layout)
stop_button.on_click(lambda x: stop())
#
camera_button = ipywidgets.Button(description='CAMERA ON',layout=button_layout)
camera_button.on_click(lambda x: toggle_camera())
#
motor_button = ipywidgets.Button(description='MOTOR OFF',layout=button_layout)
motor_button.on_click(lambda x: toggle_motor())
#
stutter_button = ipywidgets.Button(description='SLOW OFF',layout=button_layout)
stutter_button.on_click(lambda x: toggle_stutter())
#
#read only sliders
x_slider = ipywidgets.FloatSlider(min=-1, max=1, step=0.01, value=0, description='x')
y_slider = ipywidgets.FloatSlider(min=0, max=1, step=0.01, value=0, orientation='vertical', description='y')
steering_slider = ipywidgets.FloatSlider(min=-1, max=1, step=0.01, description='steering')
speed_slider = ipywidgets.FloatSlider(min=0, max=1, step=0.01, orientation='vertical', description='speed')
#
left_slider = ipywidgets.FloatSlider(min=0, max=1, step=0.01, orientation='vertical', description='L nvidia')
right_slider = ipywidgets.FloatSlider(min=0, max=1,step=0.01, orientation='vertical', description='R nvidia')
#
left_adjusted = ipywidgets.FloatSlider(min=0, max=1, step=0.01, orientation='vertical', description='L adjusted')
right_adjusted = ipywidgets.FloatSlider(min=0, max=1, step=0.01, orientation='vertical', description='R adjusted')
#
#The x and y sliders will display the predicted x, y values.
#The steering slider will display our estimated steering value.  Please remember, this value isn't the actual 
#angle of the target, but simply a value that is nearly proportional.  When the actual angle is ``0``, this 
#will be zero, and it will increase / decrease with the actual angle.  
#
red_slider = ipywidgets.FloatSlider(min=0, max=1.0, description='red')
#display(red_slider)
yellow_slider = ipywidgets.FloatSlider(min=0, max=1.0, description='yellow')
#display(yellow_slider)
green_slider = ipywidgets.FloatSlider(min=0, max=1.0, description='green')
#display(green_slider)
#
#arrange widgets in UI
status_a = ipywidgets.Textarea(value = "wait")
status_b = ipywidgets.Textarea(value = "wait")
status_c = ipywidgets.Textarea(value = "wait")
sliders = widgets.VBox([speed_gain_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider,turn_thres])
buttons = widgets.VBox([stop_button,camera_button,motor_button,stutter_button])
#buttons = widgets.VBox([start_button,stop_button,camera_button,motor_button,stutter_button])
y_sets = ipywidgets.HBox([y_slider,left_slider,right_slider,left_adjusted,right_adjusted])
x_sets = widgets.VBox([x_slider, steering_slider, ipywidgets.Label("a) action"), status_a,
                       ipywidgets.Label("b) obstacle detect"), status_b, 
                       ipywidgets.Label("c) color detect"), status_c, 
                       red_slider, yellow_slider, green_slider])
#
steer_debug = False
if not steer_debug: #hide unused controls
    turn_thres.layout.visibility = 'hidden'
    left_adjusted.layout.visibility = 'hidden'
    right_adjusted.layout.visibility = 'hidden'
#
cam_toggle = 1 #enables/disables camera, may improve performance
def toggle_camera():
    global cam_toggle
    if cam_toggle:
        camera_link.unlink()
        cam_toggle = 0
        camera_button.description = "CAMERA OFF"
    else:
        camera_link.link()
        cam_toggle = 1
        camera_button.description = "CAMERA ON"
#
mt = 0 #start/stop motor
def toggle_motor():
    global mt
    if mt: 
        mt = 0
        motor_button.description = "MOTOR OFF"
    else: 
        mt = 1
        motor_button.description = "MOTOR ON"
#       
stutter = 0 #enable/disable stutter-steps
def toggle_stutter():
    global stutter
    if stutter: 
        stutter = 0
        stutter_button.description = "SLOW OFF"
    else: 
        stutter = 1
        stutter_button.description = "SLOW ON"

#
#start processing images
def start():
    execute({'new': camera.value})
    camera.observe(execute, names='value')
#
#stop jetbot, stop camera
def stop():
    time.sleep(0.1)  # add a small sleep to make sure frames have finished processing
    camera.unobserve_all()
    robot.stop()
    camera.stop()
#
print(time.ctime(),'ok')

Mon Aug  3 13:05:19 2020 ok


### Display UI and Enable Controls
Note: to restart AUTOBUS after RED STOP BUTTON, rerun code block below

In [43]:
#make camera
try: camera.stop()
except: print('check camera')
fps = 10
camera = Camera.instance(width=224, height=224, capture_width=224, capture_height=224, fps=fps)
image_widget = widgets.Image(format='jpeg', width=224, height=224)
camera_link = traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=display_xy)
camera.start()
#
#reset vars
frame_cnt = 0 #reset frame count
time_left_station = 0 #the time (secs from epoch) it left last station
#
#!!!!! adjustable vars !!!!!
mt = 0 #0 starts jetbot without moving, hit motor button to start moving
stutter = 0 #0 disable start stop stutter movement
#robot.left_motor.alpha = 1 #scales motor speed by this, 1 is no change
#robot.right_motor.alpha = 1 
interval = 1.0/fps #set interval between execute's same as frame rate
prob_block_stop = 0.95 #only stop if prob is higher than this
station_stop_wait = 10 #stop at station for x secs
leave_station_span = 30 #the timespan (secs) it takes to leave a station (no red in sight)
f_red = 0.4 #color detection threshold
f_yellow = 0.05
f_green = 0.1
#^^^^^ adjustable vars ^^^^^
#
#run jetbot
execute({'new': camera.value}) #test single image run
start() #start jetbot
print(time.ctime(),'ok')
#
#show UI
print("vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv AUTOBUS UI vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv")
display(widgets.HBox([image_widget,sliders,buttons]))
display(ipywidgets.HBox([y_sets,x_sets]))
print("red: station stop; yellow: near station slow; green: resume normal speed")
print("^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ AUTOBUS UI ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^")
#
#IMPORTANT: The camera must be re-initialized after it stops
#Rerun this block to reinitialize camera
#First run takes a few min to start up

Mon Aug  3 13:05:20 2020 ok
vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv AUTOBUS UI vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv


red: station stop; yellow: near station slow; green: resume normal speed
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ AUTOBUS UI ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


### Debug

In [44]:
#stop()

In [45]:
#camera.stop()

In [46]:
#robot.stop()

In [34]:
#execute({'new': camera.value})
#camera.observe(execute, names='value')

In [35]:
#start()

In [36]:
steer_debug = False #False = default NVIDIA steering; True = custom; don't use if NVIDIA steering code works fine
#????? steering debug vars ????? only in effect if steer_debug = True
if steer_debug:
    turn_high = 0.4
    turn_low = 0.3
    turn_thres.value = 0.3 #only used if steer_debug=True
    fwd_high = 0.37
    fwd_low = 0.33
    interval = 0.1
if steer_debug:
    turn_thres.layout.visibility = 'visible'
    left_adjusted.layout.visibility = 'visible'
    right_adjusted.layout.visibility = 'visible'
else:
    turn_thres.layout.visibility = 'hidden'
    left_adjusted.layout.visibility = 'hidden'
    right_adjusted.layout.visibility = 'hidden'
#^^^^^ steering debug vars ^^^^^ only in effect if steer_debug = True